In [15]:

# === BLOQUE 2 – Ingeniería de Features a nivel product_id ===

# 1) (Opcional) Si no tienes tslearn instalado, descomenta la siguiente línea:
#%pip install tslearn --quiet
#%pip install holidays --quiet

# 2) Importar librerías
import pandas as pd
import numpy as np
from pathlib import Path
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset
import os

# 3) Rutas 

#BASE_DATOS     = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\datasets'
BASE_DATOS     = 'C:/Users/Elisabeth/Desktop/MAESTRIA_AUSTRAL/Labo_III/labo3-2025v/datasets/'
FEATURES_DIR   = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\features'
INTERMEDIOS   = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\intermedios'
SALIDAS       = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\output'



# 4) Función para optimizar tipos numéricos
def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df


# 5) Cargar el DataFrame limpio generado en el Bloque 1
print(f"📥 Cargando df_limpio_product_id desde {INTERMEDIOS} …")
df = pd.read_csv(os.path.join(INTERMEDIOS, "df_limpio_product_id.csv"))
df.reset_index(drop=True, inplace=True)
#Participacion productid x Cat

# 1. Asegúrate de tener las columnas requeridas:
cols = ['periodo', 'product_id', 'tn', 'cat1', 'cat2', 'cat3']
assert set(cols).issubset(df.columns), f"Faltan columnas: {set(cols) - set(df.columns)}"

# 2. Suma total por mes y cat1, cat2, cat3
df['total_cat1'] = df.groupby(['periodo', 'cat1'])['tn'].transform('sum')
df['total_cat2'] = df.groupby(['periodo', 'cat2'])['tn'].transform('sum')
df['total_cat3'] = df.groupby(['periodo', 'cat3'])['tn'].transform('sum')

# 3. Calcula la participación del product_id en cada nivel
df['share_cat1'] = df['tn'] / df['total_cat1']
df['share_cat2'] = df['tn'] / df['total_cat2']
df['share_cat3'] = df['tn'] / df['total_cat3']

# 4. (Opcional) Limpiar columnas intermedias
df.drop(columns=['total_cat1','total_cat2','total_cat3'], inplace=True)


# 5. Revisa el resultado
print(df[['periodo','product_id','tn','cat1','share_cat1','cat2','share_cat2','cat3','share_cat3']].head())


# 6) Variables de calendario y estacionalidad
print("🔄 Calculando variables de calendario y estacionales…")
df['period'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m').dt.to_period('M')

df['year']               = df['period'].dt.year
df['month']              = df['period'].dt.month
df['days_in_month']      = df['period'].dt.days_in_month
df['semester']           = ((df['month'] - 1) // 6) + 1
df['quarter']            = df['period'].dt.quarter
df['month_q']            = ((df['month'] - 1) % 3) + 1

#Orden secuencial
first_period             = df['period'].min()
df['period_ordinal']     = ((df['period'].dt.year  - first_period.year)  * 12 +
                             (df['period'].dt.month - first_period.month + 1))
# Estacionalidad global
N = df['period_ordinal'].max()  # O ponés 36 si es fijo
df['period_ordinal_sin'] = np.sin(2 * np.pi * df['period_ordinal'] / N)
df['period_ordinal_cos'] = np.cos(2 * np.pi * df['period_ordinal'] / N)

df['month_sin']          = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos']          = np.cos(2 * np.pi * df['month'] / 12)
df['quarter_sin']        = np.sin(2 * np.pi * df['quarter'] / 4)
df['quarter_cos']        = np.cos(2 * np.pi * df['quarter'] / 4)
df['is_summer']          = df['month'].isin([12,1,2]).astype(int)
df['is_winter']          = df['month'].isin([6,7,8]).astype(int)
df['pre_march']          = (df['month'] == 2).astype(int)
df['pre_october']        = (df['month'] == 9).astype(int)
print("✅ Estacionales calculadas.")

# 7) Antigüedad de producto y flag legacy
print("🔄 Calculando edad de producto y flag legacy…")
min_global = df['period_ordinal'].min()
first_p    = df.groupby('product_id')['period_ordinal'].transform('min')
df['product_age']          = df['period_ordinal'] - first_p
df['is_legacy_product']    = (first_p == min_global).astype(int)

# productos legacy con edad NaN o cap
df.loc[df['is_legacy_product'] == 1, 'product_age'] = 0
df['product_age_capped36'] = df['product_age'].clip(upper=36)
print("✅ Antigüedad y legacy calculados.")


# 8) One-Hot Encoding para cat1
#print("🔄 One-Hot Encoding de cat1…")
#cat1_dummies = pd.get_dummies(df['cat1'], prefix='cat1')
#df = pd.concat([df.drop(columns=['cat1']), cat1_dummies], axis=1)
#print("✅ cat1 codificado.")

# 9) === LAGS SIMPLES (1 a 36) ===

print("🔄 Calculando lags simples (1-36)...")
df = df.sort_values(['product_id','period'])
for lag in range(1, 37):
    df[f'tn_lag{lag}'] = df.groupby('product_id')['tn'].shift(lag)
print("✅ Lags simples calculados.")

# 10) Rolling windows con transform
print("🔄 Calculando rolling windows y derivados…")
for w in range(2, 37):
    grp = df.groupby('product_id')['tn']
    df[f'tn_roll{w}']    = grp.transform(lambda x: x.rolling(window=w, min_periods=1).mean())
    df[f'tn_rollstd{w}'] = grp.transform(lambda x: x.rolling(window=w, min_periods=1).std())
    df[f'tn_rollmin{w}'] = grp.transform(lambda x: x.rolling(window=w, min_periods=1).min())
    df[f'tn_rollmax{w}'] = grp.transform(lambda x: x.rolling(window=w, min_periods=1).max())
    df[f'tn_is_new_rollmax{w}'] = (df['tn'] == df[f'tn_rollmax{w}']).astype(int)
    df[f'tn_is_new_rollmin{w}'] = (df['tn'] == df[f'tn_rollmin{w}']).astype(int)
    df[f'tn_div_roll{w}']       = df['tn'] / (df[f'tn_roll{w}'] + 1e-6)
print("✅ Rolling completado.")


# Indicador binario de nuevo maximo y nuevo minimo
for window in range(2, 37):
    # Nuevo máximo en la ventana: 1 si el valor actual es igual al máximo móvil
    df[f'tn_is_new_rollmax{window}'] = (df['tn'] == df[f'tn_rollmax{window}']).astype(int)

    # Nuevo mínimo en la ventana: 1 si el valor actual es igual al mínimo móvil
    df[f'tn_is_new_rollmin{window}'] = (df['tn'] == df[f'tn_rollmin{window}']).astype(int)

#Ratio entre el valor actual y el promedio movil
for window in range(2, 37):
    df[f'tn_div_roll{window}'] = df['tn'] / (df[f'tn_roll{window}'] + 1e-6)

for i in range(2, 13):  # desde 2 para evitar división por cero o varianza nula
    df[f'tn_zscore_ma{i}'] = (df['tn'] - df[f'tn_roll{i}']) / (df[f'tn_rollstd{i}'] + 1e-6)


# 11) Deltas y aceleraciones
print("🔄 Calculando deltas y aceleraciones…")
for lag in range(1, 37):
    df[f'tn_delta{lag}']     = df['tn'] - df[f'tn_lag{lag}']
    df[f'tn_rel_delta{lag}'] = df[f'tn_delta{lag}'] / (df[f'tn_lag{lag}'] + 1e-6)
for lag in range(1, 12):
    df[f'tn_accel{lag}']     = df[f'tn_delta{lag}'] - df[f'tn_delta{lag+1}']
print("✅ Deltas y aceleraciones calculados.")

for i in range(2, 37):
    df[f'tn_vs_ma{i}'] = df['tn'] - df[f'tn_roll{i}']

# 1. Defino qué patrones de columnas quiero ver
#cols_delta       = [c for c in df.columns if c.startswith('tn_delta')]
#cols_rel_delta   = [c for c in df.columns if c.startswith('tn_rel_delta')]
#cols_accel       = [c for c in df.columns if c.startswith('tn_accel')]
#cols_vs_ma       = [c for c in df.columns if c.startswith('tn_vs_ma')]

# 2. Armo la lista completa de columnas de interés
#cols_interes = ['tn', 'target'] + cols_delta + cols_rel_delta + cols_accel + cols_vs_ma

# 3. Filtro por product_id y muestro las primeras filas
#df_filtrado = df.loc[df['product_id'] == 20001, cols_interes]

#df_filtrado


# 12) Slopes de tendencia con transform
print("🔄 Calculando slopes de tendencia…")
def rolling_slope(x, window):
    idx = np.arange(window)
    def _slope(arr):
        if len(arr) < window:
            arr = arr[-window:]
            idx2 = np.arange(len(arr))
            if len(arr) < 2:
                return np.nan
            return np.polyfit(idx2, arr, 1)[0]
        return np.polyfit(idx, arr, 1)[0]
    return x.rolling(window).apply(_slope, raw=True)

for window in [2, 3, 6, 9, 12, 18, 24, 36]:
    df[f'tn_trend_slope{window}'] = (
        df
        .groupby(['product_id'])['tn']
        .apply(lambda x: rolling_slope(x, window))
        .reset_index(level=[0,1], drop=True)
    )
print("✅ Slopes calculados.")


# 13) CV, agregados globales y ratios
print("🔄 Calculando CV, agregados y ratios…")
grp = df.groupby('product_id')['tn']
for w in range(2, 37):
    df[f'cv_roll{w}'] = df[f'tn_rollstd{w}'] / (df[f'tn_roll{w}'] + 1e-6)
df['tn_mean_all']   = grp.transform('mean')
df['tn_max_all']    = grp.transform('max')
df['tn_min_all']    = grp.transform('min')
df['tn_count_zero'] = grp.transform(lambda x: (x==0).sum())
df['tn_to_mean_all']= df['tn'] / (df['tn_mean_all'] + 1e-6)
df['tn_to_max_all'] = df['tn'] / (df['tn_max_all'] + 1e-6)
print("✅ Estadísticas globales y ratios calculados.")

# 14) Récords y percentil históricos
print("🔄 Agregando récords y percentil…")
df['is_all_time_high'] = (df['tn'] == df['tn_max_all']).astype(int)
df['is_all_time_low']  = (df['tn'] == df['tn_min_all']).astype(int)
df['tn_pct_all']       = df.groupby('product_id')['tn'].transform(lambda x: x.rank(pct=True))
print("✅ Récords y percentil agregados.")

#15) TN min y max de los ultimos n meses

# 1. Ordena
df = df.sort_values(['product_id','period'])

# 2. Agrupa
grp = df.groupby('product_id')['tn']

# 3. Rolling min y max en un solo bucle
for n in range(3, 37):
    df[f'tn_min_last_{n}'] = grp.transform(
        lambda s: s.rolling(window=n, min_periods=1).min()
    )
    df[f'tn_max_last_{n}'] = grp.transform(
        lambda s: s.rolling(window=n, min_periods=1).max()
    )

# 4. Comprueba algunas columnas
print([c for c in df.columns if c.startswith('tn_min_last_')][:3],
      [c for c in df.columns if c.startswith('tn_max_last_')][:3])


#16) ## Edad del producto y del cliente (feature “edad”)
# Primer periodo (nacimiento) de cada producto y cada cliente
prod_nac = (
    df
    .groupby('product_id')['period']
    .min()
    .reset_index()
    .rename(columns={'period': 'prod_start'})
)


# Suponiendo que ya tienes prod_nac y cust_nac
df = df.merge(prod_nac, on='product_id', how='left')

# Edad en meses de product_id en cada fila
df['prod_age'] = (df['period'] - df['prod_start']).apply(lambda x: x.n)


# 4. Flag de producto preexistente:
#    marca con 1 aquellos productos cuyo prod_start coincide
#    con el período más antiguo de todo el df
periodo_inicio_global = df['period'].min()
df['producto_preexistente'] = (
    df['prod_start'] == periodo_inicio_global
).astype(int)



#17) #Meses Consecutivos con tn==0
# 1. Asegúrate de ordenar por product_id y periodo
df = df.sort_values(['product_id', 'period'])

# 2. Función auxiliar que recorre la serie y cuenta ceros consecutivos
def cuenta_ceros_consecutivos(serie):
    contador = 0
    resultado = []
    for v in serie:
        if v == 0:
            contador += 1
        else:
            contador = 0
        resultado.append(contador)
    return resultado

# 3. Aplicar por grupo y transformar en nueva columna
df['meses_ceros_consec'] = df.groupby('product_id')['tn'] \
                             .transform(cuenta_ceros_consecutivos)

#Comprobar que funciona el feature
# 1. ¿Cuál es el primer mes de cada producto en el df?
#primeros = df.groupby('product_id')['period'].first()
#print(primeros.loc[20001])  # muestra el mes de 'nacimiento' de product_id 20001

# 2. ¿Y cuántos meses consecutivos de cero hay en esa serie?
#serie20001 = df[df['product_id']==20001].sort_values('period')['tn']
#print(cuenta_ceros_consecutivos(serie20001))

# 18) meses consecutivos ventas distintas de cero
# 1. Asegúrate de ordenar cronológicamente
df = df.sort_values(['product_id', 'period'])

# 2. Función que cuenta consecutivos de tn != 0
def cuenta_no_ceros(serie):
    contador = 0
    resultado = []
    for v in serie:
        if v != 0:
            contador += 1
        else:
            contador = 0
        resultado.append(contador)
    return resultado

# 3. Aplicar por grupo
df['meses_no_ceros_consec'] = (
    df
    .groupby('product_id')['tn']
    .transform(cuenta_no_ceros)
)


# 19) cantidad de meses que las ventas fueron CERO en los ultimos n meses ( n = 3, 6, 12, 18,24,30,36)

# 1. Asegúrate de tener tu df ordenado por product_id y periodo
df = df.sort_values(['product_id', 'period'])

# 2. Crea una máscara que valga 1 cuando tn == 0 y 0 en otro caso
df['zero_mask'] = (df['tn'] == 0).astype(int)

# 3. Para cada valor de n, calcula la suma en ventana móvil de tamaño n
for n in [3, 6, 12, 18,24,30,36]:
    df[f'zeros_last_{n}'] = (
        df
        .groupby('product_id')['zero_mask']
        .transform(lambda s: s.rolling(window=n, min_periods=1).sum())
    )

# 4. Opcional: elimina la máscara intermedia si ya no la necesitas
df.drop(columns=['zero_mask'], inplace=True)



#20)  Indicadores de “actividad” / “inactividad prolongada”
# Función que calcula la racha de meses consecutivos sin vender
def calcular_racha_no_ventas(serie_no_sale: pd.Series) -> pd.Series:
    """
    Dada una Serie booleana/0-1 que indica para cada mes si no hubo venta (1) o sí hubo (0),
    devuelve otra Serie del mismo tamaño donde cada posición es el número de meses consecutivos
    sin venta que lleva hasta esa fila (incluyéndola). Reinicia a 0 cuando hay venta.
    """
    racha = []
    cuenta = 0
    for valor in serie_no_sale:
        if valor == 1:
            cuenta += 1
        else:
            cuenta = 0
        racha.append(cuenta)
    return pd.Series(racha, index=serie_no_sale.index)

df = df.sort_values(['product_id','period']).copy()

# Calcular no_sale (1 si tn == 0, ó 0 si tn > 0)
df['no_sale'] = (df['tn'] == 0).astype(int)

# 4) Aplicar la función por cada grupo <product_id, customer_id>
df['no_sale_streak'] = (
    df
    .groupby(['product_id'])['no_sale']
    .apply(calcular_racha_no_ventas)
    .reset_index(level=[0,1], drop=True)
)

# 21) === EVENTOS EXTERNOS ===
df['evento_agosto2019'] = (df['periodo'] == 201906).astype(int)
df['evento_crisis_post_paso'] = df['periodo'].isin([201906, 201907]).astype(int)
df['evento_control_precios_2020'] = (df['periodo'] >= 201911).astype(int)
df['es_precios_cuidados'] = df['plan_precios_cuidados'].fillna(False).astype(int)


#===Variables Exogenas===

import pandas as pd
import requests
import holidays

# 1. Series de tipo de cambio (ARS por USD al fin de mes)
exchange_data = {
    '2017-01': 15.893, '2017-02': 15.577, '2017-03': 15.527, '2017-04': 15.344,
    '2017-05': 15.695, '2017-06': 16.089, '2017-07': 17.150, '2017-08': 17.425,
    '2017-09': 17.231, '2017-10': 17.459, '2017-11': 17.493, '2017-12': 17.723,
    '2018-01': 19.344, '2018-02': 20.148, '2018-03': 20.543, '2018-04': 20.530,
    '2018-05': 24.223, '2018-06': 27.246, '2018-07': 28.266, '2018-08': 30.879,
    '2018-09': 39.391, '2018-10': 38.075, '2018-11': 37.485, '2018-12': 38.840,
    '2019-01': 38.430, '2019-02': 39.428, '2019-03': 42.542, '2019-04': 44.354,
    '2019-05': 46.089, '2019-06': 44.955, '2019-07': 43.750, '2019-08': 54.650,
    '2019-09': 58.790, '2019-10': 61.403, '2019-11': 63.013, '2019-12': 63.012
}
exch = pd.Series(exchange_data, name='USD_ARS')
exch.index = pd.to_datetime(exch.index + '-01')

# 2. Conteo de feriados nacionales por mes
ar_holidays = holidays.country_holidays('AR', years=[2017, 2018, 2019])
holiday_counts = [
    sum(1 for d in ar_holidays if (d.year == date.year and d.month == date.month))
    for date in exch.index
]
holiday_series = pd.Series(holiday_counts, index=exch.index, name='holiday_count')

###IPC###
# 1) Leemos el Excel
file_path = os.path.join(BASE_DATOS, "IPC.xlsx")

ipc_raw = pd.read_excel(
    file_path,
    sheet_name='Indice_IPC_Nacional',               # ajusta al nombre de tu hoja
    skiprows=0,                        # si necesitas saltar filas de encabezado
    usecols=['Mes', 'IPC']  # columnas de fecha y valor IPC
)

# 2) Revisar que cargó bien
print(ipc_raw.head())
print(ipc_raw.columns.tolist())

#3) Parsear fechas y poner índice
ipc_raw['Periodo'] = pd.to_datetime(ipc_raw['Mes'], format='%Y-%m')
ipc_raw.set_index('Periodo', inplace=True)

# 4) Filtrar rango y renombrar
ipc = ipc_raw.loc['2017-01-01':'2019-12-31', 'IPC'].copy()
ipc.name = 'IPC'

# 5) Asegurar que el índice sea fin de mes
ipc.index = ipc.index.to_period('M').to_timestamp()

# Mostrar resultado
print(ipc)

# Finalmente concatenas con las series de USD_ARS y holiday_count
exog_df = pd.concat([exch, ipc, holiday_series], axis=1)
exog_df

# 1) Copia exog_df y convierte su índice datetime a Period[M]
exog_period = exog_df.copy()
exog_period.index = exog_period.index.to_period('M')

# 2) Merge directo por Period[M]
df = df.merge(
    exog_period,
    left_on='period',    # tu columna Period[M]
    right_index=True,    # índice también Period[M]
    how='left'
)

# Clusters Jerarquicos dtaidistance + SciPy

# Instala dtaidistance
%pip install dtaidistance --quiet

from dtaidistance import dtw
from scipy.cluster.hierarchy import linkage, fcluster

# 1. Prepara tu pivot tal como antes
df_ts    = df.groupby(['product_id','periodo'])['tn'].sum().reset_index()
df_pivot = df_ts.pivot(index='product_id', columns='periodo', values='tn').fillna(0)

# 2) Z-normalización de cada serie para hacerlas comparables en forma
ts_array = to_time_series_dataset(df_pivot.values)  # (n_series, n_timestamps, 1)
scaler   = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0)
ts_norm  = scaler.fit_transform(ts_array)            # cada fila → media 0, varianza 1

# 3) Crea lista de arrays 1D para dtaidistance
series_norm = [s.flatten() for s in ts_norm]


# 2. Convierte a lista de series para dtaidistance
#series = [df_pivot.loc[pid].values for pid in df_pivot.index]

#D_full = dtw.distance_matrix(series, compact=False)
#D_condensed = squareform(D_full)

# 3. Calcula la matriz de distancias DTW (compacta)
D_compact = dtw.distance_matrix_fast(series_norm, compact=True)

# 4. Calcula el linkage jerárquico sobre esa distancia
#    Puedes probar 'average', 'complete', 'single', etc.
Z = linkage(D_compact, method='average')

# 5. Corta en k clusters (p.ej. k=5)
k = 5
labels = fcluster(Z, t=k, criterion='maxclust')

# 6. Crea el DataFrame de clusters y mergea como antes
df_clusters = pd.DataFrame({
    'product_id': df_pivot.index,
    'cluster_dtw': labels
})
df_clusters['cluster_dtw'] = df_clusters['cluster_dtw'].astype('category')

#aca hacemos el merge con SF

df = df.merge(df_clusters, on='product_id', how='left')

print("✅ Clusters jerárquicos DTW agregados como categoría.")

df.shape

# Validamos posibles inf o NaN

# Lista para acumular resultados
resultados = []

for col in df.columns:
    nulos = df[col].isnull().sum()
    # Solo contamos infinitos si la columna es numérica
    if pd.api.types.is_numeric_dtype(df[col]):
        infs = np.isinf(df[col]).sum()
    else:
        infs = 0
    resultados.append({
        'column': col,
        'null_count': nulos,
        'inf_count': infs
    })

resumen = pd.DataFrame(resultados)
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
#resumen

df['target'].head()

# Ver un conteo de tipos
print(df.dtypes.value_counts())

# 1. Detectar columnas por tipo
cols_obj    = df.select_dtypes(include=['object']).columns
cols_bool   = df.select_dtypes(include=['bool']).columns
cols_dt     = df.select_dtypes(include=['datetime64[ns]']).columns
cols_period = df.columns[df.dtypes == 'period[M]']

# 2. Unir listas
cols_sel = cols_obj.tolist() + cols_period.tolist() + cols_bool.tolist() + cols_dt.tolist()

# 3. Mostrar el head sólo de esas columnas
df[cols_sel].head()

# 2. Object → category (o numérico si toca)
for c in df.select_dtypes('object').columns:
    df[c] = df[c].astype('category')


# 1. Crear las nuevas columnas a partir de prod_start
df['prod_start_ord']   = df['prod_start'].apply(lambda p: p.ordinal)
df['prod_start_year']  = df['prod_start'].dt.year
df['prod_start_month'] = df['prod_start'].dt.month

# Y luego borras únicamente esa columna original
df.drop(columns=['prod_start'], inplace=True)

# 1. Bool → int
df['plan_precios_cuidados'] = df['plan_precios_cuidados'].astype('int')

# 17) Optimizar dtypes y guardar como pkl
print("🔄 Optimizando dtypes y guardando como pickle…")
df = optimize_dtypes(df)
df = df.drop(columns=['fecha'], errors='ignore')

# ——— Aquí insertas el print de features ———
print(f"✅ Total de features: {len(df.columns)}")
#print("Listado de features:")
#for col in df.columns:
#    print("  -", col)
# ————————————————————————————————

#10) Guardado de resultados en Drive (PKL sin compresión)
print("🔄 Guardando resultados en Drive como pkl...")
df.to_pickle(
    os.path.join(FEATURES_DIR , "dataset_features_product_id.pkl"),
)

print("✅ Procesamiento completado.")
print("ℹ️ Archivo pkl guardado en:", os.path.join(FEATURES_DIR, "dataset_features_product_id.pkl"))



📥 Cargando df_limpio_product_id desde C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\intermedios …
   periodo  product_id         tn cat1  share_cat1         cat2  share_cat2  \
0   201701       20001   934.7722   HC    0.058264  ROPA LAVADO    0.116544   
1   201702       20001   798.0162   HC    0.049812  ROPA LAVADO    0.101629   
2   201703       20001  1303.3577   HC    0.056091  ROPA LAVADO    0.115016   
3   201704       20001  1069.9613   HC    0.054734  ROPA LAVADO    0.118268   
4   201705       20001  1502.2013   HC    0.063953  ROPA LAVADO    0.122996   

      cat3  share_cat3  
0  Liquido    0.324148  
1  Liquido    0.280829  
2  Liquido    0.328195  
3  Liquido    0.344682  
4  Liquido    0.355674  
🔄 Calculando variables de calendario y estacionales…
✅ Estacionales calculadas.
🔄 Calculando edad de producto y flag legacy…
✅ Antigüedad y legacy calculados.
🔄 Calculando lags simples (1-36)...
✅ Lags simples calculados.
🔄 Calculando rolling w

C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tn_rollmax{w}'] = grp.transform(lambda x: x.rolling(window=w, min_periods=1).max())
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:129: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tn_is_new_rollmax{w}'] = (df['tn'] == df[f'tn_rollmax{w}']).astype(int)
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:130: PerformanceWarning: DataFrame is highly fragmented.  This is us

✅ Rolling completado.
🔄 Calculando deltas y aceleraciones…


C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tn_delta{lag}']     = df['tn'] - df[f'tn_lag{lag}']
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:155: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'tn_rel_delta{lag}'] = df[f'tn_delta{lag}'] / (df[f'tn_lag{lag}'] + 1e-6)
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:154: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

✅ Deltas y aceleraciones calculados.
🔄 Calculando slopes de tendencia…


...

✅ Slopes calculados.
🔄 Calculando CV, agregados y ratios…


C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:210: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['tn_count_zero'] = grp.transform(lambda x: (x==0).sum())
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:211: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['tn_to_mean_all']= df['tn'] / (df['tn_mean_all'] + 1e-6)
C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:212: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

✅ Estadísticas globales y ratios calculados.
🔄 Agregando récords y percentil…


C:\Users\Elisabeth\AppData\Local\Temp\ipykernel_9504\210172110.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['tn_pct_all']       = df.groupby('product_id')['tn'].transform(lambda x: x.rank(pct=True))


✅ Récords y percentil agregados.
['tn_min_last_3', 'tn_min_last_4', 'tn_min_last_5'] ['tn_max_last_3', 'tn_max_last_4', 'tn_max_last_5']
         Mes       IPC
0 2017-01-01  101.5859
1 2017-02-01  103.6859
2 2017-03-01  106.1476
3 2017-04-01  108.9667
4 2017-05-01  110.5301
['Mes', 'IPC']
Periodo
2017-01-01    101.5859
2017-02-01    103.6859
2017-03-01    106.1476
2017-04-01    108.9667
2017-05-01    110.5301
2017-06-01    111.8477
2017-07-01    113.7852
2017-08-01    115.3819
2017-09-01    117.5719
2017-10-01    119.3528
2017-11-01    120.9940
2017-12-01    124.7956
2018-01-01    126.9887
2018-02-01    130.0606
2018-03-01    133.1054
2018-04-01    136.7512
2018-05-01    139.5893
2018-06-01    144.8053
2018-07-01    149.2966
2018-08-01    155.1034
2018-09-01    165.2383
2018-10-01    174.1473
2018-11-01    179.6388
2018-12-01    184.2552
2019-01-01    189.6101
2019-02-01    196.7501
2019-03-01    205.9571
2019-04-01    213.0517
2019-05-01    219.5691
2019-06-01    225.5370
2019-07-01  